## Step 9: 함수를 더 편리하게

우리는 자동 역전파도 구현했고 Define-by-Run도 구현했다. 하지만 여전히 조금 불편한 부분이 있기 때문에, 여기서는 그러한 문제점을 해결할 개선점 세 가지를 추가하도록 하겠다.

### 9.1 파이썬 함수로 이용하기

지금까지의 DeZero는 함수를 '파이썬 클래스'로 정의해서 사용했기 때문에 어떤 함수를 사용하기 위해서는 해당 함수 클래스의 인스턴스를 생성한 후 그 인스턴스를 호출하는 두 단계로 진행해야 했다.

```python
x = Variable(np.array(0.5))
f = Square() # 인스턴스 생성
y = f(x) # 인스턴스 호출
```

이러한 문제를 해결하기 위한 방법으로 '파이썬 함수'를 들 수 있다.

```python
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)
```

위처럼 두 단계로 나뉘어진 동작을 묶어 하나의 함수로 만들면 다음과 같이 인스턴스 생성 부분을 생략할 수 있다.

```python
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)
# y = square(exp(square(x)))

y.grad = np.array(1.0)
y.backward()
print(x.grad)
```

### 9.2 backward 메서드 간소화

기존 방식은 역전파 시마다 `y.grad = np.array(1.0)`이라는 코드를 작성해야 했다. 이번에는 이 부분을 간소화 시켜보도록 하겠다.

```python
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)
```

변수의 grad가 None이면 자동으로 미분값을 생성한다. np.ones_like(self.data) 코드는 self.data와 형상과 데이터 타입이 같은 ndarray 인스턴스를 생성한다. 이때의 값은 모두 1이다.

```python
# 간소화 된 계산
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)
```

### 9.3 ndarray만 취급하기

기본적으로 Variable의 인풋 데이터 타입은 ndarray만 사용하도록 유도하였다. 하지만 사용자의 실수로 int나 float 등 잘못된 데이터 타입이 들어올 가능성이 있다. 따라서 여기서는 ndarray 외의 다른 데이터 타입이 들어오면 에러를 내도록 하겠다.

```python
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f'{type(data)} type is not supported.')
                
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)
```

```python
x = Variable(np.array(1.0)) # OK
x = Variable(None) # OK

x = Variable(1.0) # No
```

하지만 여기서 주의할 점이 있다. 다음 예시를 살펴보자.

```python
x = np.array(1.0) # 0차원 ndarray
y = x ** 2
print(type(x))
print(type(y))
```

<class 'numpy.ndarray'>


<class 'numpy.float64'>

여기서 x는 0차원의 ndarray인데, 이를 제곱한 y는 numpy.float64이다. 이는 numpy의 특성에 의한 것으로, 0차원 ndarray 인스턴스를 사용하여 계산하면 결과의 데이터 타입이 np.float64 혹은 np.float32 등으로 달라지게 된다.

따라서 다음과 같은 helper function을 추가하여 이를 해결해주도록 하겠다.

```python
def as_array(x):
    if np.array(x):
        return np.array(x)
    return x
```

`np.isscalar` 함수는 입력 데이터가 numpy.float64 같은 스칼라 타입인지 확인해주는 함수이다. 이를 통해 x가 스칼라 타입인지 여부를 쉽게 확인할 수 있으며, as_array 함수를 통해 스칼라 입력을 ndarray로 변환해줄 수 있다.

```python
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
```

이와 같이 순전파의 결과인 y를 Variable로 감쌀 때 as_array()를 이용한다. 이렇게 하여 출력 결과인 output이 항상 ndarray 인스턴스가 되도록 보장할 수 있게 되었다.